In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
 
spark_jars_packages = ",".join(
    [
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0"
    ]
)
 
spark = (
    SparkSession.builder
    .master("local")
    .appName('test connect to kafka')
    .config("spark.jars.packages", spark_jars_packages)
    .getOrCreate()
)


23/09/18 02:57:41 WARN Utils: Your hostname, fhmgi3sbi8tfqsopje66 resolves to a loopback address: 127.0.1.1; using 10.128.0.32 instead (on interface eth0)
23/09/18 02:57:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c82dec45-056f-4d1d-8be3-c81e4562c6b6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://r

23/09/18 02:57:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
 
df = (spark.read
      .format('kafka')
      .option('kafka.bootstrap.servers', 'rc1b-2erh7b35n4j4v869.mdb.yandexcloud.net:9091')
      .option('kafka.security.protocol', 'SASL_SSL')
      .option('kafka.sasl.mechanism', 'SCRAM-SHA-512')
      .option('kafka.sasl.jaas.config',
              'org.apache.kafka.common.security.scram.ScramLoginModule required username=\"de-student\" password=\"ltcneltyn\";')
      .option("subscribe", "persist_topic")
      .load())


In [3]:

schema = StructType([
    StructField("subscription_id", IntegerType()),
    StructField("name", StringType()),
    StructField("description", StringType()),
    StructField("price", DoubleType()),
    StructField("currency", StringType())
])
 
df = (df
      .withColumn('value', f.col('value').cast(StringType()))
      .withColumn('key', f.col('key').cast(StringType()))
      .withColumn('event', f.from_json(f.col('value'), schema))
      .selectExpr('event.*', '*').drop('event')
      )
 
df.printSchema()


root
 |-- subscription_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



+---------------+----------+-------------------------------------------------------------+-----+--------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+---------+------+-----------------------+-------------+
|subscription_id|name      |description                                                  |price|currency|key  |value                                                                                                                                                       |topic        |partition|offset|timestamp              |timestampType|
+---------------+----------+-------------------------------------------------------------+-----+--------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+---------+------+-----------------------+-

In [4]:

df.show()

+---------------+----------+--------------------+-----+--------+-----+--------------------+-------------+---------+------+--------------------+-------------+
|subscription_id|      name|         description|price|currency|  key|               value|        topic|partition|offset|           timestamp|timestampType|
+---------------+----------+--------------------+-----+--------+-----+--------------------+-------------+---------+------+--------------------+-------------+
|           null|      null|                null| null|    null| test|               test2|persist_topic|        0|     0|2023-04-24 10:24:...|            0|
|           null|      null|                null| null|    null|test2|               test3|persist_topic|        0|     1|2023-04-24 10:25:...|            0|
|              1|      Free|   Free subscription|  0.0|     USD|    1|{"subscription_id...|persist_topic|        0|     2|2023-05-01 17:19:...|            0|
|              2|Individual|$14.95/mo. Save $...|14.